In [19]:
import xarray as xr
print(xr.__version__)


2025.7.1


In [3]:
import xarray as xr

def process_all_quarters(file_path: str):
    """
    Memuat file NetCDF harian dan melakukan agregasi ke kuartalan
    untuk seluruh rentang waktu data.

    Args:
        file_path (str): Path menuju file .nc.
        
    Returns:
        xarray.Dataset: Dataset yang berisi data kuartalan, atau None jika error.
    """
    try:
        ds = xr.open_dataset(file_path)
        if 't' in ds.coords:
            ds = ds.rename({'t': 'time'})

        # Agregasi ke data kuartalan untuk seluruh 10 tahun
        quarterly_data = ds.resample(time='Q').mean()
        
        return quarterly_data

    except Exception as e:
        print(f"Terjadi error saat memproses {file_path}: {e}")
        return None

In [5]:
# Daftar file dan variabel yang akan diproses
# Struktur ini (list of dictionaries) sudah baik
files_to_process = [
    {'sst': 'dataset/Philippines/cmems_mod_glo_bgc-pft_anfc_0.25deg_P1D-m_1754231774728.nc'},
    {'chrol_a': 'dataset/Philippines/cmems_mod_glo_phy-cur_anfc_0.083deg_P1D-m_1754231512886.nc'},
    {'sea_current_speed': 'dataset/Philippines/cmems_mod_glo_phy-thetao_anfc_0.083deg_P1D-m_1754231610874.nc'}
]

# Dictionary untuk menyimpan semua data kuartalan yang sudah diproses
all_quarterly_data = {}

# --- PERUBAHAN PADA LOOP DI BAWAH INI ---

# Loop untuk memproses setiap dictionary di dalam list
for data_dict in files_to_process:
    # Loop untuk mengambil key (var_name) dan value (path) dari setiap dictionary
    for var_name, path in data_dict.items():
        print(f"Memproses {path} untuk seluruh 10 tahun...")
        
        # Panggil fungsi dan simpan hasilnya
        result = process_all_quarters(path)
        
        if result is not None:
            all_quarterly_data[var_name] = result

# --- Verifikasi Hasil ---
print("\n--- Data kuartalan berhasil diproses dan disimpan ---")

# Contoh: Cek data suhu (sst)
if 'sst' in all_quarterly_data:
    sst_data_10_years = all_quarterly_data['sst']
    print("\nInformasi data SST Kuartalan (10 Tahun):")
    print(sst_data_10_years)
    # Outputnya akan menunjukkan ~40 time steps (4 kuartal x 10 tahun)

Memproses dataset/Philippines/cmems_mod_glo_bgc-pft_anfc_0.25deg_P1D-m_1754231774728.nc untuk seluruh 10 tahun...
Memproses dataset/Philippines/cmems_mod_glo_phy-cur_anfc_0.083deg_P1D-m_1754231512886.nc untuk seluruh 10 tahun...


c:\CodeTools\Projek\Gemastik\.venv\Lib\site-packages\xarray\groupers.py:509: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  self.index_grouper = pd.Grouper(
c:\CodeTools\Projek\Gemastik\.venv\Lib\site-packages\xarray\groupers.py:509: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  self.index_grouper = pd.Grouper(


Memproses dataset/Philippines/cmems_mod_glo_phy-thetao_anfc_0.083deg_P1D-m_1754231610874.nc untuk seluruh 10 tahun...

--- Data kuartalan berhasil diproses dan disimpan ---

Informasi data SST Kuartalan (10 Tahun):
<xarray.Dataset> Size: 199kB
Dimensions:    (time: 16, depth: 1, latitude: 69, longitude: 45)
Coordinates:
  * depth      (depth) float32 4B 0.494
  * latitude   (latitude) float32 276B 4.0 4.25 4.5 4.75 ... 20.5 20.75 21.0
  * longitude  (longitude) float32 180B 116.0 116.2 116.5 ... 126.5 126.8 127.0
  * time       (time) datetime64[ns] 128B 2021-12-31 2022-03-31 ... 2025-09-30
Data variables:
    chl        (time, depth, latitude, longitude) float32 199kB nan ... 0.09347
Attributes:
    Conventions:       CF-1.11
    title:             daily mean fields from Global Ocean Biogeochemistry An...
    institution:       Mercator Ocean
    producer:          CMEMS - Global Monitoring and Forecasting Centre
    credit:            E.U. Copernicus Marine Service Information (CMEMS

c:\CodeTools\Projek\Gemastik\.venv\Lib\site-packages\xarray\groupers.py:509: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  self.index_grouper = pd.Grouper(


In [6]:
!c:/CodeTools/Projek/Gemastik/.venv/Scripts/python.exe -m pip install xarray pandas numpy netCDF4

In [8]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

def analyze_nc_file(file_path: str, variable_name: str):
    """
    Menganalisis file NetCDF dengan memuatnya, menampilkan statistik,
    dan membuat visualisasi peta.

    Args:
        file_path (str): Path menuju file .nc Anda.
        variable_name (str): Nama variabel yang ingin dianalisis (misal: 'sst', 'chl').
    """
    try:
        # 1. Muat Dataset
        print(f"--- Menganalisis File: {file_path} ---")
        ds = xr.open_dataset(file_path)

        # 2. Eksplorasi Struktur Data
        print("\n[Struktur Data]")
        print(ds)

        # Pilih variabel data yang akan dianalisis
        data_var = ds[variable_name]

        # 3. Hitung Statistik Dasar
        print(f"\n[Statistik untuk Variabel: '{variable_name}']")
        # Menggunakan .item() untuk mendapatkan nilai tunggal dari array
        mean_val = np.nanmean(data_var).item()
        max_val = np.nanmax(data_var).item()
        min_val = np.nanmin(data_var).item()
        std_val = np.nanstd(data_var).item()

        print(f"  - Rata-rata (Mean) : {mean_val:.2f}")
        print(f"  - Nilai Maksimum   : {max_val:.2f}")
        print(f"  - Nilai Minimum    : {min_val:.2f}")
        print(f"  - Standar Deviasi  : {std_val:.2f}")

        # 4. Buat Visualisasi Peta (rata-rata sepanjang waktu)
        print("\n[Membuat Visualisasi Peta...]")
        plt.figure(figsize=(12, 8))
        
        # Hitung rata-rata data di seluruh dimensi waktu
        # dan buat plot 2D (lintang vs bujur)
        data_var.mean(dim='time').plot(cmap='jet')
        
        plt.title(f'Peta Rata-rata {variable_name.upper()}')
        plt.xlabel('Bujur (Longitude)')
        plt.ylabel('Lintang (Latitude)')
        plt.grid(True)
        plt.show()

    except FileNotFoundError:
        print(f"ERROR: File tidak ditemukan di '{file_path}'")
    except KeyError:
        print(f"ERROR: Variabel '{variable_name}' tidak ditemukan. Variabel yang tersedia: {list(ds.data_vars.keys())}")
    except Exception as e:
        print(f"Terjadi error: {e}")


# --- CARA MENGGUNAKAN FUNGSI ---
# Ganti dengan path file dan nama variabel Anda
file_to_analyze = 'dataset/Philippines/cmems_mod_glo_bgc-pft_anfc_0.25deg_P1D-m_1754231774728.nc'
variable_to_analyze = 'sst'

analyze_nc_file(file_to_analyze, variable_to_analyze)

--- Menganalisis File: dataset/Philippines/cmems_mod_glo_bgc-pft_anfc_0.25deg_P1D-m_1754231774728.nc ---

[Struktur Data]
<xarray.Dataset> Size: 17MB
Dimensions:    (time: 1377, depth: 1, latitude: 69, longitude: 45)
Coordinates:
  * time       (time) datetime64[ns] 11kB 2021-11-01 2021-11-02 ... 2025-08-08
  * depth      (depth) float32 4B 0.494
  * latitude   (latitude) float32 276B 4.0 4.25 4.5 4.75 ... 20.5 20.75 21.0
  * longitude  (longitude) float32 180B 116.0 116.2 116.5 ... 126.5 126.8 127.0
Data variables:
    chl        (time, depth, latitude, longitude) float32 17MB ...
Attributes:
    Conventions:       CF-1.11
    title:             daily mean fields from Global Ocean Biogeochemistry An...
    institution:       Mercator Ocean
    producer:          CMEMS - Global Monitoring and Forecasting Centre
    credit:            E.U. Copernicus Marine Service Information (CMEMS)
    contact:           https://marine.copernicus.eu/contact
    references:        http://marine.copern

In [14]:
import geopandas as gpd

# 1. Muat Shapefile Anda
# Ganti dengan path ke file .shp Anda
shapefile_path = 'dataset/Philippines/Province Border/phl_admbnda_adm2_psa_namria_20231106.shp'
gdf = gpd.read_file(shapefile_path)

# 2. Simpan sebagai file GeoPackage tunggal
# File ini akan berisi semua informasi dalam satu file.
gdf.to_file('philippines_provinces.gpkg', driver='GPKG')

print("Shapefile successfully converted to a single GeoPackage file: 'philippines_provinces.gpkg'")

Shapefile successfully converted to a single GeoPackage file: 'philippines_provinces.gpkg'


In [18]:
import geopandas as gpd

gdf = gpd.read_file("dataset/Philippines/Province Border/phl_admbnda_adm2_psa_namria_20231106.shp")

# See all column names
print(gdf.columns.tolist())

# Optional: See first few rows
print(gdf.head())


['ADM2_EN', 'ADM2_PCODE', 'ADM1_EN', 'ADM1_PCODE', 'ADM0_EN', 'ADM0_PCODE', 'date', 'validOn', 'validTo', 'Shape_Leng', 'Shape_Area', 'ADM2ALT1EN', 'AREA_SQKM', 'geometry']
        ADM2_EN ADM2_PCODE                     ADM1_EN ADM1_PCODE  \
0  Ilocos Norte    PH01028    Region I (Ilocos Region)       PH01   
1    Ilocos Sur    PH01029    Region I (Ilocos Region)       PH01   
2      La Union    PH01033    Region I (Ilocos Region)       PH01   
3    Pangasinan    PH01055    Region I (Ilocos Region)       PH01   
4       Batanes    PH02009  Region II (Cagayan Valley)       PH02   

             ADM0_EN ADM0_PCODE       date    validOn validTo  Shape_Leng  \
0  Philippines (the)         PH 2022-11-09 2023-11-06     NaT    2.850087   
1  Philippines (the)         PH 2022-11-09 2023-11-06     NaT    4.152632   
2  Philippines (the)         PH 2022-11-09 2023-11-06     NaT    2.403317   
3  Philippines (the)         PH 2022-11-09 2023-11-06     NaT    7.246192   
4  Philippines (the)       